In [5]:
import nltk
#nltk.download()
from __future__ import unicode_literals
from hazm import *
import re

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#importing data from file
df=pd.read_excel('failure_description.xlsx')

In [8]:
#looking to dataframe
df.iloc[:,1][0]

'بازو بیرونی شماره21و6 گریسکاری نمیشود'

In [ ]:
#func for removing english character
def remove_eng(x):
    return re.sub('[0-9a-zA-Z]',' ',x)

In [ ]:
#mapping function to dataframe
df['Problem_Description'] = df.iloc[:,1].map(lambda x: remove_eng(x))

In [ ]:
#looking
df

In [ ]:
#useing HAZM for normalizing sentenses
normalizer = Normalizer(token_based=True)
def norm(x):
    return normalizer.normalize(x)

In [ ]:
#sample test 
norm(df.iloc[:,1][0])

In [ ]:
#mapping function to dataframe
df['Normalized_description'] = df.iloc[:,1].map(lambda x: norm(x))

In [ ]:
#useing HAZM for tokenizing
def word_tok(x):
    return word_tokenize(x)
df['word_tokenize'] = df.iloc[:,2].map(lambda x: word_tok(x))

In [ ]:
#looking
df.iloc[:,3][0]

In [ ]:
#removing empty item and space from begin and end of words
def remove_space(x):
    remove_space = [item.strip() for item in x]
    delete_empty = [item for item in remove_space if item.strip()]
    return delete_empty
    
df['removed_space'] = df.iloc[:,3].map(lambda x: remove_space(x))        

In [ ]:
#save cleaned dataframe to excel file
#df.to_excel('temp.xlsx')

In [ ]:
#sample test
a=[' یتبنیمشس سینش  ','','نسشیت    ']
print(a)
remove_space(a)

In [ ]:
#removing stopwords
stop_words_df=pd.read_excel('stop_words.xlsx')
stop_words = stop_words_df.iloc[:,0].to_list()
def remove_stopwords(x):
    output=[]
    for item in x:
        if item not in stop_words:
            output.append(item)
    return output

In [ ]:
#mapping function to dataframe
df['removed_stopwords'] = df.iloc[:,4].map(lambda x: remove_stopwords(x))

In [ ]:
#making sentence by joining
df['sentens'] = df.iloc[:,5].map(lambda x: ' '.join(x))

In [ ]:
#making new dataframe by selecting some colume from old dataframe
temp=df[['Cause_of_failure','sentens','removed_stopwords']].copy()

In [ ]:
#removing NaN
final_df=temp[temp['Cause_of_failure'].notnull()]

In [ ]:
#looking
final_df

In [ ]:
#changing categure value by 0,1,2,....
res = dict(zip(final_df.Cause_of_failure.value_counts().keys().to_list(),range(27)))
final_df.Cause_of_failure.replace(res,inplace = True)

In [ ]:
#changed by this values
res.items()

In [ ]:
#reindexing dataframe
final_df.reset_index(inplace= True,drop=True)

In [ ]:
#looking
final_df.info()

In [ ]:
#save cleaned dataframe to excel file
#final_df.to_excel('f_data.xlsx')

In [ ]:
#Vectorizing data by sklearn "Bag Of Words"
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
#appling model to dataframe
bow = cv.fit_transform(final_df.iloc[:,1]).toarray()

In [ ]:
bow.shape

In [ ]:
#Vectorizing data by sklearn "TF-idf"
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#making class
tf = TfidfVectorizer()

In [ ]:
#training data
tf.fit(final_df.iloc[:,1])

In [ ]:
#looking to vocabulary
print(tf.fit(final_df.iloc[:,1]).vocabulary_)

In [ ]:
#making vectors by TF-idf
X=tf.transform(final_df.iloc[:,1]).toarray()

In [ ]:
y=final_df['Cause_of_failure']

In [ ]:
X[0].sum()

In [ ]:
#spliting data to train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)

In [ ]:
X_train.shape

In [ ]:
#training model 
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB().fit(X_train,y_train)

In [ ]:
#predicting 
y_pred=model.predict(X_test)

In [ ]:
#calculating accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

In [ ]:
#calculating accuracy
y_predict=model.predict(X_train)
accuracy_score(y_predict,y_train)

In [ ]:
#Indexing words of corpus by tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
#Indexing
max_words = 500
tkn = Tokenizer(num_words=max_words)
tkn.fit_on_texts(final_df.iloc[:,1])
seq = tkn.texts_to_sequences(final_df.iloc[:,1])
print(len(seq))
for i in range(10):
    print(seq[i],'<----->',final_df.iloc[:,1][i])

In [ ]:
#calculating Maximum Len
max_len=0
for item in seq:
    if len(item)>max_len:
        max_len=len(item)
print(max_len)

In [ ]:
#making list with equal len
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_docs = pad_sequences(seq, padding = 'pre', maxlen= max_len)
print(padded_docs)

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
embedding_vector_features = 10
model = Sequential()
model.add(Embedding(max_words, embedding_vector_features, input_length=max_len))
model.add(LSTM(256))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(padded_docs,y,batch_size=128,epochs=10, validation_split=0.2)

In [ ]:
#prediction 
text='جک نشتی دارد'.strip().split()
print(text)
pr=pad_sequences(tkn.texts_to_sequences([text]), padding = 'pre', maxlen= max_len)
print(pr)
model.predict(pr)

In [ ]:
#bag of words useing sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer=CountVectorizer()
vocabulary=vectorizer.fit(final_df.iloc[:,1])

In [ ]:
X= vectorizer.transform(final_df.iloc[:,1])

In [ ]:
X.toarray()

In [ ]:
print(vocabulary.get_feature_names())

In [ ]:
for item in final_df.iloc[:,1]:
    if item.find(' یکسر') != -1:
        print(item)
        print(final_df[final_df['sentens']== item])

In [ ]:
final_df

In [ ]:
from googletrans import Translator

In [ ]:
tr = Translator()

In [ ]:
tr.translate('contents', dest='en')